In [ ]:
!pip install --user pykafka

In [ ]:
from pykafka import KafkaClient
client = KafkaClient(hosts="188.185.79.229:9092")

topic = client.topics['email_alert']

In [ ]:
consumer = topic.get_simple_consumer()
for message in consumer:
    print(message.value)

In [ ]:
!wget http://central.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.3.1/spark-streaming-kafka-0-8-assembly_2.11-2.3.1.jar

In [ ]:
from pyspark.streaming.kafka import KafkaUtils
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.streaming.kafka import TopicAndPartition
import json

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
def sendingAlert(subject,count):
    return [subject,count,count>10]
spark.udf.register("sendingAlert", sendingAlert)

In [ ]:
ssc = StreamingContext(sc, 5) #change frequency of getting data
ssc.checkpoint("_checkpoint")

In [ ]:
kafkaStream =KafkaUtils.createDirectStream(ssc, ["email_alert"],
                                           {
                                               "metadata.broker.list":"188.185.79.229:9092",
                                               "auto.offset.reset":"smallest"
                                           }
                                          )

In [ ]:
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

In [ ]:
emails_dstream = parsed.map(lambda email: [email['subject'],email['timestamp'],1])\
                        .reduceByKeyAndWindow(lambda x,y: x+y, 10, 5)
# emails_dstream.pprint()
# benchmark=emails_dstream.map(lambda x:[x[0],x[1],x[1]>30])
benchmark=emails_dstream.map(lambda x: sendingAlert(x[0],x[1]))
# structuredOutput.foreachRDD(rdd => 
#   import sparkSession.implicits._
#   val df = rdd.toDF()
#   df.write.format("parquet").mode("append").save(s"$workDir/$targetFile")
# })
benchmark.pprint()
ssc.start()
ssc.awaitTerminationOrTimeout(10)


In [ ]:
ssc.stop()

In [ ]:
!pip install --user kafka-python

In [ ]:
from kafka import KafkaConsumer
import json
try:
    print('Welcome to parse engine')
    consumer = KafkaConsumer('cmsweb_logs',
                             bootstrap_servers='monit-kafka.cern.ch:9092',
#To subscribe since the first message that has been produced to kafka producer
#                              auto_offset_reset='earliest',
                             enable_auto_commit=False,
                             group_id=None,
                             value_deserializer=lambda m: json.loads(m)
                            )
    for message in consumer:
        print(message)
except Exception as e:
    print(e)
    # Logs the error appropriately.
    pass